# Introducción
Actualmente contamos con una cantidad ingente de información, de la cual gran parte se encuentra en forma de texto. Para las personas es fácil leerla y encontrar relaciones entre las palabras que conforman un texto, sin embargo para las computadoras es una tarea compleja. Debido a esto surgen áreas de las ciencias de la computación que se encargan de la extracción de información en texto mediante diversas técnicas. En este sentido, la información mutua es una técnica se puede aplicar en el descubrimiento de las asociaciones de
palabras.

El propósito de este documento es realizar el descubrimiento de asociaciones de palabras para los conjuntos de datos proporcionados utilizando la medida de información mutua.

# Desarrollo
Para la solución de este problema se utilizó Python.

In [1]:
!pip install unidecode

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import string
import unidecode
import re
import itertools
from nltk.util import ngrams
import math

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
es_stop_words = set(stopwords.words('spanish'))

### Descripción de los datos
Para realizar esta tarea se tienen 3 conjuntos de datos que están contienen tweets de México, España y Venezuela del mes de marzo de 2020, con 1000000, 1000000 y 300000 filas respectivamente. Cada conjunto está formado por 3 columnas: created_at, id y text. En este caso, la columna que nos interesa es text.

In [ ]:
mx_tweets = pd.read_json("/content/drive/MyDrive/ProcesamientoInformacion/MX_1M.json", encoding="utf-8", lines = True)
es_tweets = pd.read_json("/content/drive/MyDrive/ProcesamientoInformacion/ES_1M.json", encoding="utf-8", lines = True)
vz_tweets = pd.read_json("/content/drive/MyDrive/ProcesamientoInformacion/VE_300K.json", encoding="utf-8", lines = True)

In [ ]:
mx_tweets.head(10)

En la celda anterior podemos ver las primeras 10 filas del conjunto de datos de tweets de México.

### Implementación
A continuación se describen las funciones usadas y se proporcionan ejemplos de su funcionamiento

In [ ]:
def preprocess(text):
    no_urls = re.sub(r"http\S+", "", text)
    #remover @usuarios y #hashtags
    without_users = re.sub('@[\w]+','',no_urls)
    without_users_and_hashtags = re.sub('#[\w]+','',without_users)
    #remover acentos
    unaccented_string = unidecode.unidecode(without_users_and_hashtags)
    #convertir a minuscula y remover signos de puntuacion
    text_lower = unaccented_string.lower().translate(str.maketrans('', '', string.punctuation))
    #limitar los characteres repetidos consecutivos
    #obtenemos los tokens
    text_tokens = word_tokenize(text_lower)
    #filtramos las stopwords de los tokens obtenidos
    tokens_without_sw = [word for word in text_tokens if not word in es_stop_words]
    tokens_without_dups = [limit_dups(token) for token in tokens_without_sw]
    return tokens_without_dups

def limit_dups(s, limit=2):
    max_vs = (''.join(itertools.islice(g, limit)) for k, g in itertools.groupby(s))
    components = ([s[:l + 1] for l in range(len(s))] for s in max_vs)
    return [''.join(letters) for letters in itertools.product(*components)][-1]

La función preprocess recibe una cadena de texto y se encarga de realizar el  preprocesamiento en el siguiente orden: remover urls, usuarios y hashtags, después se quitan los acentos, se convierte el texto a minúsculas y se remueven los signos de puntuación, se convierte el texto a tokens para posteriormente removerle las stop-words y finalmente se limitan los carácteres redundantes.

En la siguiente celda veremos el resultado de la función de preprocesamiento aplicada al texto: "Felizzzzzzzz jueves!!! en Torre Reforma https://t.co/EV46yOUOEx"

In [ ]:
preprocess("Felizzzzzzzz jueves!!! en Torre Reforma https://t.co/EV46yOUOEx")

In [ ]:
def fill_dict(text_array):
    my_dictionary = {}
    for word in text_array:
        for item in word:
            if item in my_dictionary:
                my_dictionary[item] = my_dictionary[item] + 1
            else:
                my_dictionary[item] = 1
    return my_dictionary

filter_dict = lambda my_dict, limit = 5: {k: v for k, v in my_dict.items() if v > limit}

def calculate_mutual_information(data_dict, data_bigram_dict, limit = 50):
  mx_mutual_information = {}
  for item in data_bigram_dict:
      mx_mutual_information[item] = mutual_information(item, item[0], item[1], data_bigram_dict, data_dict)
  return sorted(mx_mutual_information.items(), key=lambda x:x[1], reverse=True)[:limit]

La función fill_dict crea un diccionario a partir de un arreglo de strings usando cada palabra como llave. Si la llave ya está, se aumenta su cuenta, sino se le asigna un 1. Esta función se usa para contar la frecuencia de los tokens.

La función filter_dict nos permite filtrar un diccionario por valor. Esta función se usa para filtrar los tokens que tengan una frecuencia menor a cierto valor. En este caso, 5 es un valor predeterminado.

La función calculate_mutual_information obtiene las medidas de información mutua para todos los bigramas de un conjunto. Devuelve, de forma predeterminada, los 50 primeros ordenados según su índice de información mutua de mayor a menor.

In [ ]:
def mutual_information(xy, x, y, data1, data2):
    pxy = data1[xy]/len(data2)
    px = data2[x]/len(data2)
    py = data2[y]/len(data2)
    return math.log2(pxy/((px)*(py)))

La función mutual_information nos permite calcular la información mutua de acuerdo a la siguiente fórmula: $I\left ( x_{i};y_{i} \right ) = \log_2{\frac{p\left ( x_{i},y_{i} \right )}{p\left (  x_{i}\right) p\left (  y_{i}\right)}}$, donde $p\left(x_{i}\right)$ y $p\left(y_{i}\right)$ se obtienen de las frecuencias relativas de los datos

In [ ]:
mx_tweets['processed_text']= mx_tweets['text'].map(preprocess)
es_tweets['processed_text']= es_tweets['text'].map(preprocess)
vz_tweets['processed_text']= vz_tweets['text'].map(preprocess)

### Resultados
Primero aplicamos el preprocesamiento al campo text de cada uno de los conjuntos y lo asignamos a una nueva columna llamada processed_text

In [ ]:
mx_tweets["bigrams"] = mx_tweets["processed_text"].apply(lambda x: list(ngrams(x, 2)))
es_tweets["bigrams"] = es_tweets["processed_text"].apply(lambda x: list(ngrams(x, 2)))
vz_tweets["bigrams"] = vz_tweets["processed_text"].apply(lambda x: list(ngrams(x, 2)))

In [ ]:
mx_tweets.head(10)

Luego obtenemos los bigramas a partir del texto procesado

In [ ]:
mx_dictionary = fill_dict(mx_tweets['processed_text'])
es_dictionary = fill_dict(es_tweets['processed_text'])
vz_dictionary = fill_dict(vz_tweets['processed_text'])

In [ ]:
mx_bigram_dictionary = fill_dict(mx_tweets['bigrams'])
es_bigram_dictionary = fill_dict(es_tweets['bigrams'])
vz_bigram_dictionary = fill_dict(vz_tweets['bigrams'])

Después se llenan los diccionarios con las frecuencias de las palabras y bigramas

In [ ]:
mx_dictionary_freq_filtered = filter_dict(mx_dictionary)
es_dictionary_freq_filtered = filter_dict(es_dictionary)
vz_dictionary_freq_filtered = filter_dict(vz_dictionary)

In [ ]:
mx_bigrams_dictionary_freq_filtered = filter_dict(mx_bigram_dictionary)
es_bigrams_dictionary_freq_filtered = filter_dict(es_bigram_dictionary)
vz_bigrams_dictionary_freq_filtered = filter_dict(vz_bigram_dictionary)

Posteriormente se filtran los diccionarios para quedarnos solo con aquellos que tengan una frecuencia mayor a 5

Finalmente, llamamos a la función que calcula la información mutua para los datos de cada conjunto y obtenemos una lista de tuplas de las 50 asociaciones más importantes junto a su índice de información mutua.

In [ ]:

mx_mutual_information = calculate_mutual_information(mx_dictionary_freq_filtered, mx_bigrams_dictionary_freq_filtered)
mx_mutual_information


La lista anterior es la lista de las 50 asociaciones más frecuentes para el conjunto de datos de México. Podemos hallar asociaciones como Duane Cochran, prismas basálticos, Lindsay Lohan o Hakuna Matata, etc.

In [ ]:

es_mutual_information = calculate_mutual_information(es_dictionary_freq_filtered, es_bigrams_dictionary_freq_filtered)
es_mutual_information


La lista anterior es la lista de las 50 asociaciones más frecuentes para el conjunto de datos de España. Podemos encontrar asociaciones como Hugh Jackman, Greys Anatomy, Childish Gambino, etc.

In [ ]:

vz_mutual_information = calculate_mutual_information(vz_dictionary_freq_filtered, vz_bigrams_dictionary_freq_filtered)
vz_mutual_information


La lista anterior es la lista de las 50 asociaciones más frecuentes para el conjunto de datos de Venezuela. Podemos hallar asociaciones como Peaky Blinders, hipertensa diabética, vivito coleando, feng shui, etc.

Como último experimento, filtramos los tokens y bigramas que tengan una frecuencia mayor a 100 y nuevamente calculamos el índice de información mutua

In [ ]:
mx_dictionary_freq_filtered_100 = filter_dict(mx_dictionary, 100)
mx_bigrams_dictionary_freq_filtered_100 = filter_dict(mx_bigram_dictionary, 100)
mx_mutual_information_100 = calculate_mutual_information(mx_dictionary_freq_filtered_100, mx_bigrams_dictionary_freq_filtered_100)
mx_mutual_information_100

Es interesante ver que hay asociaciones como ('neumonia', 'atipica'), ('pronta', 'recuperacion'), ('cubre', 'bocas'), ('gel', 'antibacterial'). Cabe destacar que en marzo de 2020 se declaró que (COVID-19) se podía caracterizar como una pandemia.

# Conclusiones
La medida de información mutua es muy útil para hallar asociaciones entre palabras en un texto. Sin embargo, su cálculo y las asociaciones que encontremos estarán directamente relacionadas con el preprocesamiento que le apliquemos al texto. Por otra parte, debe reconocerse su importancia es otras áreas como procesamiento de lenguaje natural o recuperación de información.